In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple
import numpy as np
# from geneticalgorithm import geneticalgorithm as ga
import pandas as pd
from collections import Counter
from tqdm import tqdm
import time
from Bio.SeqUtils import MeltingTemp
from Bio import SeqIO
from plotly import graph_objects as go
import json
from imp import reload
# import primer_selection
# reload(primer_selection)
# import testing
# reload(testing)


/tmp/ipykernel_1803897/859605501.py:18: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [9]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO
from Bio.Blast import NCBIXML
import subprocess

In [2]:
def complement_sequence(dna_sequence):
    trans = str.maketrans('ATCG', 'TAGC')
    return dna_sequence.upper().translate(trans)

# print(complement_sequence("ATGCGTA"))
# # Example usage:
# # dna_sequence = "ATGCGTA"
# # complement_sequence = complement_sequence(dna_sequence)
# # print(complement_sequence)  # Output: TACGCAT
# #%%
def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)


TACGCAT


In [36]:
def calculate_mismatches(seq1, seq2):
    return sum(1 for a, b in zip(seq1, seq2) if a != b)

def find_alignments(sequence, genome, target_site):
    alignments = []
    seq_len = len(sequence)
    genome_len = len(genome)
    
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i+seq_len]
        mismatches = calculate_mismatches(sequence, subseq)
        binding_location = (i, i+seq_len)
        distance_from_target = min(abs(i - target_site), abs(i + seq_len - target_site))
        
        alignment_info = {
            'binding_location': binding_location,
            'mismatches': mismatches,
            'distance_from_target': distance_from_target
        }
        alignments.append(alignment_info)
    
    return alignments

# Usage
sequence = 'ATG'
genome = 'ATGCGTATGACGTAG'
target_site = 7  # Assume the target site is at position 7
alignments = find_alignments(sequence, genome, target_site)

for alignment in alignments:
    binding_location = alignment['binding_location']
    mismatches = alignment['mismatches']
    distance_from_target = alignment['distance_from_target']
    print(f'Binding location: {binding_location}, Mismatches: {mismatches}, Distance from target: {distance_from_target}')



Binding location: (0, 3), Mismatches: 0, Distance from target: 4
Binding location: (1, 4), Mismatches: 3, Distance from target: 3
Binding location: (2, 5), Mismatches: 2, Distance from target: 2
Binding location: (3, 6), Mismatches: 3, Distance from target: 1
Binding location: (4, 7), Mismatches: 2, Distance from target: 0
Binding location: (5, 8), Mismatches: 3, Distance from target: 1
Binding location: (6, 9), Mismatches: 0, Distance from target: 1
Binding location: (7, 10), Mismatches: 3, Distance from target: 0
Binding location: (8, 11), Mismatches: 3, Distance from target: 1
Binding location: (9, 12), Mismatches: 1, Distance from target: 2
Binding location: (10, 13), Mismatches: 3, Distance from target: 3
Binding location: (11, 14), Mismatches: 2, Distance from target: 4
Binding location: (12, 15), Mismatches: 2, Distance from target: 5


In [ ]:
def calculate_mismatches(seq1, seq2):
    return sum(1 for a, b in zip(seq1, seq2) if a != b)

def find_alignments(sequence, genome, target_site):
    alignments = []
    seq_len = len(sequence)
    genome_len = len(genome)
    
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i+seq_len]
        mismatches = calculate_mismatches(sequence, subseq)
        binding_location = (i, i+seq_len)
        distance_from_target = min(abs(i - target_site), abs(i + seq_len - target_site))
        
        alignment_info = {
            'binding_location': binding_location,
            'mismatches': mismatches,
            'distance_from_target': distance_from_target
        }
        alignments.append(alignment_info)
    
    return alignments

# Usage
sequence = 'ATG'
genome = 'ATGCGTATGACGTAG'
target_site = 7  # Assume the target site is at position 7
alignments = find_alignments(sequence, genome, target_site)

for alignment in alignments:
    binding_location = alignment['binding_location']
    mismatches = alignment['mismatches']
    distance_from_target = alignment['distance_from_target']
    print(f'Binding location: {binding_location}, Mismatches: {mismatches}, Distance from target: {distance_from_target}')

In [76]:
ref_genome = '/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/MTB-h37rv_asm19595v2-eg18.fa'

def genome_size(fasta_file):
    total_length = 0
    with open(fasta_file, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                total_length += len(line.strip())
    return total_length

def extract_sequence_from_fasta(start_pos, end_pos, padding = 150, fasta_file= 'MTB-h37rv_asm19595v2-eg18.fa', sequence_id='Chromosome'):
    """
    Extracts a subsequence from a FASTA file based on the given sequence ID, start position, and end position.
    """
    # Iterate over the sequences in the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Check if the current sequence ID matches the desired sequence ID
        
        if record.id == sequence_id:
            # Extract the subsequence based on the start and end positions
            subsequence = record.seq[start_pos-padding:end_pos+padding]
            return str(subsequence)  # Return the subsequence as a string
    # If the sequence ID is not found, return None
    return None

extract_sequence_from_fasta(0, genome_size(ref_genome))

'CGAGCCGTTGCCGGTAGGTTGCGGCTGGTTATCGACGGTACTGTCCACATTTGTGGATAGCCATGTGGACAGTTCACCTGCCCACAACAACGGTTGTAGCTCGACCCGGAACCAAGACCCGGAACTAACGAGAACCAGGGAGATACGTCG'

In [74]:
genome_size(ref_genome)

4411532

In [78]:
genome_size(ref_genome)

4411532

In [80]:
import primer_selection

TACGCAT


In [ ]:
primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)

In [83]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)  # Adds 0 if dinucleotide not found
    return dg


def is_good_primer_candidate(primer, genome, target_site, max_mismatches, binding_location_range, max_distance_from_target, min_tm, max_dg):
    primer_len = len(primer)
    genome_len = len(genome)
    
    for i in range(genome_len - primer_len + 1):
        subseq = genome[i:i+primer_len]
        mismatches = calculate_mismatches(primer, subseq)
        distance_from_target = min(abs(i - target_site), abs(i + primer_len - target_site))
        
        # Check if the alignment meets the thresholds
        within_mismatch_threshold = mismatches <= max_mismatches
        within_binding_location_range = binding_location_range[0] <= i <= binding_location_range[1]
        within_distance_threshold = distance_from_target <= max_distance_from_target
        
        # Evaluate binding strength
        tm = simplified_tm(subseq)
        dg = simplified_dg(subseq)
        strong_binding = tm >= min_tm and dg <= max_dg
        
        # If a strong alternative binding site is found, return False
        summary = within_mismatch_threshold + within_binding_location_range + within_distance_threshold + strong_binding
        # if within_mismatch_threshold and within_binding_location_range and within_distance_threshold and strong_binding:
        if summary > 3:
            return False
    return True  # No strong alternative binding sites found

# Usage
primer = 'CGAACTCGAGGCTGCCTACT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
target_site = 7
max_mismatches = 2
binding_location_range = (0, 15)
max_distance_from_target = 50
min_tm = 50  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

is_good_candidate = is_good_primer_candidate(primer, genome, target_site, max_mismatches, binding_location_range, max_distance_from_target, min_tm, max_dg)
print(is_good_candidate)


True


In [ ]:
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        tm = simplified_tm(sequence)
        dg = simplified_dg(sequence)

        within_similarity_threshold = calculate_similarity(sequence, subseq, similarity_threshold)
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

## Below is the code used from testing alternative binding

In [140]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm=60, max_dg=-10):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        if calculate_similarity(subseq, sequence)>similarity_threshold:
            tm = simplified_tm(subseq)
            dg = simplified_dg(subseq)
            
            within_tm_threshold = tm >= min_tm
            within_dg_threshold = dg <= max_dg

            if within_tm_threshold and within_dg_threshold:
                print(genome[i:i + seq_len])
                count += 1
                if count > 1:
                    return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'CGAACTCGAGGCTGCCTACT'
# primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
result = has_multiple_binding_sites(primer, genome, 81)
print(result)  # Output: True or False


CGAACTCGAGGCTGCCTACT
CGGACTCGAGGGTGCCTTCT
True


In [132]:
def has_multiple_binding_sites(sequence, genome):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        if subseq == sequence:
            print(genome[i:i + seq_len])
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'CGAACTCGAGGCTGCCTACT'
# primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
result = has_multiple_binding_sites(primer, genome)
print(result)  # Output: True or False


CGAACTCGAGGCTGCCTACT
CGAACTCGAGGCTGCCTACT
True


In [103]:
def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def has_multiple_binding_sites(sequence, genome, similarity_threshold):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        similarity = calculate_similarity(sequence, subseq)
        if similarity >= similarity_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
# primer = 'CGAACTCGAGGCTGCCTACT'
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 91  # For example, 90% similarity
result = has_multiple_binding_sites(primer, genome, similarity_threshold)
print(result)  # Output: True or False


False


In [104]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        similarity = calculate_similarity(sequence, subseq)
        tm = simplified_tm(subseq)
        dg = simplified_dg(subseq)

        # Check if the alignment meets the thresholds
        within_similarity_threshold = similarity >= similarity_threshold
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 70  # For example, 90% similarity
min_tm = 40  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

result = has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg)
print(result)  # Output: True or False


False


In [130]:
def calculate_similarity(seq1, seq2, similarity_threshold):
    required_matches = len(seq1) * similarity_threshold / 100
    matches = 0
    for a, b in zip(seq1, seq2):
        if a == b:
            matches += 1
        if matches >= required_matches:
            return True
    return False    

def simplified_tm(seq):
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def precompute_dinucleotide_params(genome):
    unique_dinucleotides = set(genome[i:i+2] for i in range(len(genome) - 1))
    tm_params = {dinuc: simplified_tm(dinuc) for dinuc in unique_dinucleotides}
    dg_params = {dinuc: simplified_dg(dinuc) for dinuc in unique_dinucleotides}
    return tm_params, dg_params

tm_params, dg_params = precompute_dinucleotide_params(genome)

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        tm = simplified_tm(sequence)
        dg = simplified_dg(sequence)

        within_similarity_threshold = calculate_similarity(sequence, subseq, similarity_threshold)
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 70  # For example, 90% similarity
min_tm = 40  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

result = not has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg)
print(result)  # Output: True or False

True


In [108]:
tm_params, dg_params = precompute_dinucleotide_params(genome)


In [118]:
tm = sum(tm_params[genome[i:i+2]] for i in range(len(primer) - 1))


In [119]:
tm

122

In [116]:
tm_params

{'CA': 6,
 'AC': 6,
 'TG': 6,
 'GG': 8,
 'TT': 4,
 'CT': 6,
 'CC': 8,
 'GA': 6,
 'AT': 4,
 'AA': 4,
 'GC': 8,
 'GT': 6,
 'AG': 6,
 'CG': 8,
 'TC': 6,
 'TA': 4}

In [4]:
def gc_content(sequence):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    gc_content_percentage = (gc_count / total_length) * 100
    return gc_content_percentage

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
gc_percentage = gc_content(sequence)
print(f'GC Content: {gc_percentage:.2f}%')


GC Content: 60.00%


In [3]:
import math

def calculate_tm_nearest_neighbor(sequence, na_concentration):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    percent_gc = (gc_count / total_length) * 100
    tm = 81.5 + 16.6 * math.log10(na_concentration) + 0.41 * percent_gc - 675 / total_length
    return tm

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
na_concentration = 0.05  # Example sodium ion concentration in mol/L
tm = calculate_tm_nearest_neighbor(sequence, na_concentration)
print(f'Melting Temperature (Tm): {tm:.2f}°C')


Melting Temperature (Tm): 50.75°C


In [5]:
def calculate_tm(sequence):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    at_count = sequence.count('A') + sequence.count('T')
    tm = (gc_count * 4) + (at_count * 2)
    return tm

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
tm = calculate_tm(sequence)
print(f'Melting Temperature (Tm): {tm}°C')


Melting Temperature (Tm): 64°C


In [2]:
spol = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/raw.githubusercontent.com_GaryNapier_spolpred_master_data_spoligotypes.csv')

In [4]:
spol.columns

Index(['id', 'pct_reads_mapped', 'num_reads_mapped', 'median_coverage',
       'main_lineage', 'sublineage', 'octal_spoligotype', 'binary_spoligotype',
       'spacer1', 'spacer2', 'spacer3', 'spacer4', 'spacer5', 'spacer6',
       'spacer7', 'spacer8', 'spacer9', 'spacer10', 'spacer11', 'spacer12',
       'spacer13', 'spacer14', 'spacer15', 'spacer16', 'spacer17', 'spacer18',
       'spacer19', 'spacer20', 'spacer21', 'spacer22', 'spacer23', 'spacer24',
       'spacer25', 'spacer26', 'spacer27', 'spacer28', 'spacer29', 'spacer30',
       'spacer31', 'spacer32', 'spacer33', 'spacer34', 'spacer35', 'spacer36',
       'spacer37', 'spacer38', 'spacer39', 'spacer40', 'spacer41', 'spacer42',
       'spacer43'],
      dtype='object')

In [11]:
# qwer = pd.read_csv('/mnt/storage7/jody/philippines/projects/all_philippines/tbprofiler_new/variants.txt')
s7_variants = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/variants.txt')

In [12]:
full_data = pd.read_csv('/mnt/storage10/jody/projects/variant_dump/variants.csv')


In [24]:
full_data.head()

,sample_id,genome_pos,gene,change,freq,type,sublin,drtype,drugs
0,ERR9787047,6446,gyrB,p.Ala403Ser,1.0,missense_variant,lineage6.1.1,Sensitive,NaN
1,ERR9787047,7362,gyrA,p.Glu21Gln,1.0,missense_variant,lineage6.1.1,Sensitive,NaN
2,ERR9787047,7585,gyrA,p.Ser95Thr,1.0,missense_variant,lineage6.1.1,Sensitive,NaN
3,ERR9787047,8493,gyrA,p.Leu398Phe,1.0,missense_variant,lineage6.1.1,Sensitive,NaN
4,ERR9787047,9143,gyrA,c.1842T>C,1.0,synonymous_variant,lineage6.1.1,Sensitive,NaN


In [25]:
full_data

,sample_id,genome_pos,gene,change,freq,type,sublin,drtype,drugs
0,ERR9787047,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
1,ERR9787047,7362,gyrA,p.Glu21Gln,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
2,ERR9787047,7585,gyrA,p.Ser95Thr,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
3,ERR9787047,8493,gyrA,p.Leu398Phe,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
4,ERR9787047,9143,gyrA,c.1842T>C,1.000000,synonymous_variant,lineage6.1.1,Sensitive,NaN
...,...,...,...,...,...,...,...,...,...
7420991,ERR718460,4328399,ethA,c.-926C>T,0.986486,upstream_gene_variant,lineage2.2.1,Other,NaN
7420992,ERR718460,4338365,whiB6,p.Cys53Arg,1.000000,missense_variant,lineage2.2.1,Other,NaN
7420993,ERR718460,4338595,whiB6,c.-75delG,1.000000,upstream_gene_variant,lineage2.2.1,Other,NaN
7420994,ERR718460,4407588,gid,c.615A>G,1.000000,synonymous_variant,lineage2.2.1,Other,NaN


In [17]:
s7_variants['sample_id'].unique().tolist()

(750,)

In [ ]:
full_data['sample_id'].unique().tolist()

In [21]:
overlap = set(s7_variants['sample_id'].unique().tolist()).intersection(set(full_data['sample_id'].unique().tolist()))


In [ ]:
full_data[full_data['change']=='p.Ala403Ser']

In [7]:
full_data[full_data['change']=='p.Ala403Ser']

,sample_id,genome_pos,gene,change,freq,type,sublin,drtype,drugs
0,ERR9787047,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
141,SRR8902561,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
2273,ERR2815611,6446,gyrB,p.Ala403Ser,0.985075,missense_variant,La1.1,Sensitive,NaN
2336,ERR3439434,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
3706,SRR14782548,6446,gyrB,p.Ala403Ser,0.976190,missense_variant,La1.2.BCG,Other,NaN
...,...,...,...,...,...,...,...,...,...
7414917,ERR2512732,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
7415030,ERR192047,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
7415606,ERR4192560,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
7415792,SRR1735567,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.2.BCG,Other,NaN


In [5]:
full_data.shape

(7420996, 9)

In [ ]:
full_data

In [19]:
import random

def generate_random_gene_seq(length):
    bases = ['A', 'T', 'C', 'G']
    return ''.join(random.choice(bases) for _ in range(length))

random_gene_seq = generate_random_gene_seq(300)
# print(f"Random Gene Sequence of Length 100: {random_gene_seq}")


In [23]:
random_gene_seq

'CGTCTGTACATGCCGCTCTATTACAATTATTCCACAAGGGTCAAGGGGCCGCTGCGCTCGGAACGCTCGTGATAGCGCTATCTCAACGCTATTACGGTAGATTCGACTTAAACACATACCTAAGCGCAAGGTAGCACGGCGGGCGTCTTTTAAACTTGGGCATCGCGACCACCGGCCAATTACGGTTTAGCTATCGTCTTACTTAGGCTATAGAGCGCTTTGCCAGAACAAAGGAAAGCGTAAGGATAACCACGGTTCGATGTTGATGGGCGACGACAGGTAGCTCGATCCTCCACTCCG'

In [22]:
import primer3

# Define sequence arguments
seq_args = {
    'SEQUENCE_ID': 'example',
    'SEQUENCE_TEMPLATE': random_gene_seq,
    'SEQUENCE_INCLUDED_REGION': [15, len(random_gene_seq) - 15]
}

# Define global arguments
global_args = {
    'PRIMER_TASK': 'generic',
    'PRIMER_PICK_LEFT_PRIMER': 1,
    'PRIMER_PICK_INTERNAL_OLIGO': 1,
    'PRIMER_PICK_RIGHT_PRIMER': 1,
    'PRIMER_OPT_SIZE': 20,
    'PRIMER_MIN_SIZE': 18,
    'PRIMER_MAX_SIZE': 25,
    'PRIMER_NUM_RETURN': 5,
    'PRIMER_MIN_TM': 57.0,
    'PRIMER_MAX_TM': 63.0,
    'PRIMER_MIN_GC': 20.0,
    'PRIMER_MAX_GC': 80.0,
    'PRIMER_MAX_POLY_X': 100,
    'PRIMER_INTERNAL_MAX_POLY_X': 100,
    'PRIMER_SALT_MONOVALENT': 50.0,
    'PRIMER_DNA_CONC': 50.0,
    'PRIMER_MAX_NS_ACCEPTED': 0,
    'PRIMER_MAX_SELF_ANY': 12,
    'PRIMER_MAX_SELF_END': 8,
    'PRIMER_PAIR_MAX_COMPL_ANY': 12,
    'PRIMER_PAIR_MAX_COMPL_END': 8
}

# Run the primer design
designs = primer3.design_primers(seq_args, global_args)
designs = bindings.design_primers(seq_args, global_args)

# Output results
for key, value in designs.items():
    print(f"{key}: {value}")


PRIMER_LEFT_EXPLAIN: considered 1488, GC content failed 5, low tm 540, high tm 553, high hairpin stability 11, ok 379
PRIMER_RIGHT_EXPLAIN: considered 1488, low tm 458, high tm 490, ok 540
PRIMER_INTERNAL_EXPLAIN: considered 2635, GC content failed 5, low tm 1501, high tm 384, high hairpin stability 26, ok 719
PRIMER_PAIR_EXPLAIN: considered 24, unacceptable product size 18, ok 6
PRIMER_LEFT_NUM_RETURNED: 5
PRIMER_RIGHT_NUM_RETURNED: 5
PRIMER_INTERNAL_NUM_RETURNED: 5
PRIMER_PAIR_NUM_RETURNED: 5
PRIMER_PAIR: [{'PENALTY': 0.42132891917782445, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 119, 'PRODUCT_TM': 83.74517309519567}, {'PENALTY': 0.44012224215788365, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 204, 'PRODUCT_TM': 86.62122351536374}, {'PENALTY': 0.44066222716770653, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 205, 'PRODUCT_TM': 86.7326295411887}, {'PENALTY': 0.4442396702864926, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 202, 'P

In [12]:
# Print to validate
print("Start of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][0])
print("Length of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][1])
print("Total length of SEQUENCE_TEMPLATE: ", len(seq_args['SEQUENCE_TEMPLATE']))

# Validate bounds
if seq_args['SEQUENCE_INCLUDED_REGION'][0] + seq_args['SEQUENCE_INCLUDED_REGION'][1] > len(seq_args['SEQUENCE_TEMPLATE']):
    print("Error: SEQUENCE_INCLUDED_REGION is out of bounds")


Start of SEQUENCE_INCLUDED_REGION:  1
Length of SEQUENCE_INCLUDED_REGION:  70
Total length of SEQUENCE_TEMPLATE:  71


In [3]:
seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCCACTGCCGATCACGACGATGTTCTTAGCGTCGTAGTCGAGGTCCTCGGGCCAGTGCTGCGGATGGATGATCGGCCCGACGAAATCCTCCGAGCCGGCGAATCTCGGCGAGTAGCCCTCGTCGTAGTTGTAGTAGCCGCTGCACAGAAAGAGGAATTCGCAGGTGAGGGCGCTGAGCGTGCCGTGGCTTTGGATGTGAACGGTCCAGCGGTTTTCCGCGGTCGACCAATCGGCACTGATCACCTTGTGGTGGAACCGGATATGCCTGTCGATTCCATACATGGCCGCGGTGCTCTTGACGTACTCGAGGATGGGCTTGCCGTCGGCGATCGCCTGCCGTCCGGTCCAGGGACGGAATCGGAAACCTAGCGTGTACATGTCGGAGTCGGAGCGAATTCCGGGATAACGGAACAAATCCCAGGTGCCGCCCATGGATTCCCGCTTTTCCAGGATGGCGTAGCTCTTGGTCGGGCAACGGTCCTGCAGGTGCCAGGCCGCGCTGACACCGGAGATTCCAGCGCCCACGATGACAACGTCGAGGTGCTCGGTCATGGATCCACGCTATCAACGTAATGTCGAGGCCGTCAACGAGATGTCGACACTATCGACACGTAGTAAGCTGCCAGGGTGACCACCTCCGCGGCCAGTCAGGCTTCGCTGCCTAGGGGCCGGCGCACCGCGCGGCCGTCCGGCGACGATCGTGAACTGGCGATCCTCGCCACCGCCGAGAACCTTCTCGAGGACCGTCCGCTGGCCGATATCTCGGTCGACGATCTGGCCAAGGGCGCCGGTATCTCGAGGCCGACGTTCTACTTCTATTTCCCATCCAAGGAAGCGGTGCTGCTGACCCTGCTGGACCGGGTGGTCAATCAAGCCGACATGGCCCTACAGACCCTTGCCGAGAATCCCGCCGACACCGACCGCGAGAACATGTGGCGCACCGGGATCAACGTGTTCTTCGAGACATTCGGGTCGCACAAGGCGGTAACCCGAGCCGGTCAGGCCGCCAGGGCAACCAGTGTCGAAGTCGCCGAACTGTGGTCGACGTTTATGCAGAAGTGGATCGCCTACACGGCCGCCGTGATCGACGCCGAACGCGACCGAGGCGCGGCGCCGCGCACCCTGCCGGCCCATGAACTGGCCACAGCGCTCAACCTGATGAACGAGCGGACGCTGTTCGCGTCATTCGCCGGCGAACAGCCCTCGGT'
len(seq)

1500

In [8]:
seq[205:230]

'CACGATGTAGGTGGGTGAGCGCTGC'

In [ ]:
ACTGCCGATCACGACGATGT

In [13]:
left_seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'
len(left_seq)

295

In [10]:
left_seq

'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'

In [12]:
primer_selection.extract_sequence_from_fasta(4327174, 4327174+20, padding=0)

'AACCGGATATGCCTGTCGAT'

In [15]:
primer_selection.extract_sequence_from_fasta(4327129, 4327129+20, padding=0)


'CGGTTTTCCGCGGTCGACCA'

In [14]:
4327129 + 295

4327424

In [36]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence):
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        if query_sequence in sequence:
            start_index = sequence.index(query_sequence)
            end_index = start_index + len(query_sequence)
            return f"Sequence found in {record.id} from position {start_index} to {end_index}"
    return "Sequence not found in the FASTA file"

# Example usage
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "ATCATCAACGGGACCGAGCG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)


Sequence found in Chromosome from position 760286 to 760306


In [40]:
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "AGTCCACCGACAAGACGCTG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)

Sequence found in Chromosome from position 760365 to 760385


In [35]:
4328229-350

4327879

In [41]:
find_sequence_location(fasta_file, 'CCGGTTCAGGCTTCACCACA')

'Sequence found in Chromosome from position 13 to 33'

In [45]:
from Bio import SeqIO

def find_sequence_location(query_sequence, position, fasta_file):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        print("Primer Sequence not found in FASTA file.")
        return 0,0

In [50]:
find_sequence_location('GCCTAAACCGGCGCTAAACC', 4326008, fasta_file='/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa')


(4325973, 4325993)

In [28]:
primer_selection.extract_sequence_from_fasta(4326924, 4326944, padding=0)


'ACTGCCGATCACGACGATGT'

In [25]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence, position):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        return "Sequence not found in the FASTA file"

In [29]:
query_sequence ='ACTGCCGATCACGACGATGT'
# query_sequence = primer_selection.reverse_complement_sequence(query_sequence)
find_sequence_location(fasta_file, query_sequence, 4327932)[0]
# print(result)

4326924

In [ ]:
4327129 4327629

In [ ]:
4327129

In [20]:
[[4326879, 4327879],
 [2282776, 2283776],
 [2285958, 2286958],
 [4246734, 4247734],
 [4244193, 4245193],
 [2280899, 2281899],
 [2152912, 2153912],
 [2284954, 2285954],
 [2747141, 2748141],
 [1834840, 1835840],
 [2150990, 2151990],
 [2277174, 2278174],
 [3067958, 3068958],
 [2147188, 2148188],
 [2138550, 2139550],
 [2281712, 2282712],
 [2283527, 2284527],
 [1416371, 1417371],
 [3840393, 3841393],
 [3073502, 3074502],
 [2149884, 2150884],
 [2276296, 2277296],
 [2279554, 2280554],
 [2287239, 2288239],
 [2141412, 2142412],
 [2140159, 2141159],
 [2148199, 2149199],
 [2144518, 2145518],
 [3086947, 3087947],
 [2128861, 2129861]]

4326629